Failure case of regularization
========================

In [ ]:
import sys
import os
sys.path.append(os.path.realpath(".."))
from preamble import *

sns.set()
sns.set(font_scale=1.1)
recompute = True
fontsize = 18
log = False

In [ ]:
weights = pd.read_csv(os.path.join(basename, "weights.csv"), index_col=0)["weight"].values

In [ ]:
reg_losses = []
for i in range(len(weights)):
    reg_losses.append(pd.read_csv(os.path.join(basename, f"loss_reg_{i}.csv"), index_col=0))

In [ ]:
smooth_loss = pd.read_csv(os.path.join(basename, "loss_smooth.csv"), index_col=0)

In [ ]:
save_steps = pd.read_csv(os.path.join(basename, "save_steps.csv"), index_col=0)["save_steps"].values

In [ ]:
names = ["smooth", *[f"reg_{i}"for i in range(len(weights))]]
imgs = []

for i, name in enumerate(names):
    wireframes = []
    for j in save_steps:
        img_name = os.path.join(basename, f"{name}_{j:05d}_wireframe.png")
        obj_name = os.path.join(basename, f"{name}_{j:05d}.ply")
        if not os.path.exists(img_name) or recompute:
            blender_render(obj_name, 
                           basename, 
                           14,
                           14,
                           res=100,
                           wireframe=True,
                           ours=(i==0),
                           baseline=(i!=0))
        wireframes.append(plt.imread(img_name))
    imgs.append(wireframes)

In [ ]:
save_steps[-1] = 25000

In [ ]:
base_size = 3
n_rows = len(reg_losses)+1
n_cols = len(save_steps)+1
aspect = 1
fontsize = 24
h,w,_ = imgs[0][0].shape
widths = np.ones(n_cols)
widths[-1] = 1.5

fig = plt.figure(1, figsize=(n_cols * base_size* aspect, n_rows * base_size*1), constrained_layout=True)
gs = fig.add_gridspec(n_rows,  n_cols, wspace=0.075,hspace=0.0, width_ratios=widths)


with sns.axes_style('white'):
    for j, step in enumerate(save_steps):
        ax = fig.add_subplot(gs[0, j])
        ax.imshow(imgs[0][j][:, (w-h)//2:(h-w)//2], interpolation='none')
        ax.axes.get_xaxis().set_ticklabels([])
        ax.axes.get_yaxis().set_ticklabels([])
        ax.axes.get_xaxis().set_ticks([])
        ax.axes.get_yaxis().set_ticks([])
        sns.despine(ax=ax, left=True, bottom=True)
        if j == 0:
            ax.set_ylabel(rf"Ours ($\lambda$=99)", fontsize=fontsize)

with sns.axes_style('darkgrid'):
    ax = fig.add_subplot(gs[0, -1])
    ax.plot(1e2*smooth_loss["reg_loss"].values, label=r"Laplacian loss ($\times 10^2$)")
    ax.plot(smooth_loss["im_loss"].values, label="Rendering loss")
    if log:
        ax.set_yscale('log')
    ax.set_ylabel("Loss", fontsize=2*fontsize//3)
    ax.set_xlabel("Steps", fontsize=2*fontsize//3)
    ax.legend(fontsize=2*fontsize//3)
    ax.set_yscale('log')
    ax.set_ylim((5e-6,1))
for i in range(len(reg_losses)):
    with sns.axes_style('white'):
        for j, step in enumerate(save_steps):
            ax = fig.add_subplot(gs[i+1, j])
            ax.imshow(imgs[i+1][j][:, (w-h)//2:(h-w)//2], interpolation='none')
            ax.axes.get_xaxis().set_ticklabels([])
            ax.axes.get_yaxis().set_ticklabels([])
            ax.axes.get_xaxis().set_ticks([])
            ax.axes.get_yaxis().set_ticks([])
            sns.despine(ax=ax, left=True, bottom=True)
            if i == len(reg_losses)-1:
                ax.set_title(fr"${save_steps[j]}$", y=-0.5, fontsize=fontsize)
            if j == 0:
                ax.set_ylabel(rf"$\lambda = {int(weights[i])}$", fontsize=fontsize)
        
    with sns.axes_style('darkgrid'):
        ax = fig.add_subplot(gs[i+1, -1])
        ax.plot(1e2*reg_losses[i]["reg_loss"].values, label=r"Laplacian loss ($\times 10^2$)")
        ax.plot(reg_losses[i]["im_loss"].values, label="Rendering loss")
        ax.set_yscale('log')
        if log:
            ax.set_yscale('log')
        ax.set_ylabel("Loss", fontsize=2*fontsize//3)
        ax.set_xlabel("Steps", fontsize=2*fontsize//3)
        ax.set_ylim((5e-6,1))
        if i == len(reg_losses)-1:
            ax.set_title("Loss", y=-0.5, fontsize=fontsize)

arrow = matplotlib.patches.FancyArrowPatch(
    (0.05,-0.02), (0.75,-0.02), transform=fig.transFigure,fc='black', mutation_scale = 40.)
fig.patches.append(arrow)

fig.suptitle("Iteration", fontsize=fontsize, y=-0.05, x=0.4)
plt.savefig(os.path.join(basename, "reg_fail.pdf"), format='pdf', dpi=300, bbox_inches='tight', pad_inches=0.05)